# Imports

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
!pip install tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from skimage.transform import resize
from tensorflow.keras.utils import plot_model
import pathlib
import imageio
import glob
import PIL
import nibabel as nib
import os
from tkinter import Tcl
import cv2
import tensorflow_docs.vis.embed as embed
import tensorflow_addons.layers as tfal
from keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,Conv2DTranspose,LeakyReLU,Activation,Concatenate,Add
from scipy import ndimage
import shutil
import json
import tensorflow.keras.layers as L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class InstanceNormalization(tf.keras.layers.Layer):
    def __init__(self, epsilon=1e-5):
        super(InstanceNormalization, self).__init__()
        self.epsilon = epsilon

    def build(self, input_shape):
        self.scale = self.add_weight(
            name='scale',
            shape=input_shape[-1:],
            initializer=tf.random_normal_initializer(1., 0.02),
            trainable=True)
        self.offset = self.add_weight(
            name='offset',
            shape=input_shape[-1:],
            initializer='zeros',
            trainable=True)

    def call(self, x):
        mean, variance = tf.nn.moments(x, axes=[1, 2], keepdims=True)
        inv = tf.math.rsqrt(variance + self.epsilon)
        normalized = (x - mean) * inv
        return self.scale * normalized + self.offset


class SqueezeAttention():
    def __init__(self):
        self.model = self.squeeze_attention_unet()

    def conv_block(self, x, num_filters):
        x = L.Conv2D(num_filters, 3, padding="same")(x)
        x = tfal.InstanceNormalization(axis=-1)(x)
        x = L.Activation("relu")(x)

        x = L.Conv2D(num_filters, 3, padding="same")(x)
        x = tfal.InstanceNormalization(axis=-1)(x)
        x = L.Activation("relu")(x)

        return x

    def se_block(self, x, num_filters, ratio=8):
        se_shape = (1, 1, num_filters)
        se = L.GlobalAveragePooling2D()(x)
        se = L.Reshape(se_shape)(se)
        se = L.Dense(num_filters // ratio, activation="relu", use_bias=False)(se)
        se = L.Dense(num_filters, activation="sigmoid", use_bias=False)(se)
        se = L.Reshape(se_shape)(se)
        x = L.Multiply()([x, se])
        return x

    def encoder_block(self, x, num_filters):
        x = self.conv_block(x, num_filters)
        x = self.se_block(x, num_filters)
        p = L.MaxPool2D((2, 2))(x)
        return x, p

    def decoder_block(self, x, s, num_filters):
        x = L.UpSampling2D(interpolation="bilinear")(x)
        x = L.Concatenate()([x, s])
        x = self.conv_block(x, num_filters)
        x = self.se_block(x, num_filters)
        return x

    def squeeze_attention_unet(self, input_shape=(256, 256, 3)):
        """ Inputs """
        inputs = L.Input(input_shape)

        """ Encoder """
        s1, p1 = self.encoder_block(inputs, 64)
        s2, p2 = self.encoder_block(p1, 128)
        s3, p3 = self.encoder_block(p2, 256)
        s4, p4 = self.encoder_block(p3, 256)

        b1 = self.conv_block(p4, 512)
        b1 = self.se_block(b1, 512)

        """ Decoder """
        d = self.decoder_block(b1, s4, 256)
        d1 = self.decoder_block(d, s3, 256)
        d2 = self.decoder_block(d1, s2, 128)
        d3 = self.decoder_block(d2, s1, 64)

        """ Outputs """
        outputs = L.Conv2D(3, (1, 1), activation='tanh')(d3)

        """ Model """

        model = Model(inputs, outputs, name="Squeeze-Attention-UNET")
        return model

class ResNet():
    def __init__(self):
        self.model = self.resnet_generator()

    def resnet_block(self, n_filters, input_layer):
        # weight initialization
        init = RandomNormal(stddev=0.02)
        # first layer convolutional layer
        g = Conv2D(n_filters, (3, 3), padding='same', kernel_initializer=init)(input_layer)
        g = tfal.InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)
        # second convolutional layer
        g = Conv2D(n_filters, (3, 3), padding='same', kernel_initializer=init)(g)
        g = tfal.InstanceNormalization(axis=-1)(g)
        # concatenate merge channel-wise with input layer
        g = Concatenate()([g, input_layer])
        return g

    def resnet_generator(self, image_shape=(256, 256, 3), n_resnet=7):
        # weight initialization
        init = RandomNormal(stddev=0.02)
        in_image = Input(shape=image_shape)
        g = Conv2D(64, (7, 7), padding='same', kernel_initializer=init)(in_image)
        g = tfal.InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)
        g = Conv2D(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
        g = tfal.InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        g = Conv2D(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
        g = tfal.InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        for _ in range(n_resnet):
            g = self.resnet_block(256, g)

        g = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
        g = tfal.InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        g = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer=init)(g)
        g = tfal.InstanceNormalization(axis=-1)(g)
        g = Activation('relu')(g)

        g = Conv2D(3, (7, 7), padding='same', kernel_initializer=init)(g)
        g = tfal.InstanceNormalization(axis=-1)(g)
        out_image = Activation('tanh')(g)
        # define model
        model = Model(in_image, out_image)
        return model


class unet_model():
    def __init__(self):
        self.model = self.unet_generator()

    def downsample(self, filters, size, apply_norm=True):
        initializer = tf.random_normal_initializer(0., 0.02)
        result = tf.keras.Sequential()
        result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                          kernel_initializer=initializer, use_bias=False))
        if apply_norm:
            result.add(InstanceNormalization())
        result.add(tf.keras.layers.LeakyReLU())
        return result

    def upsample(self, filters, size, apply_dropout=False):
        initializer = tf.random_normal_initializer(0., 0.02)
        result = tf.keras.Sequential()
        result.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                                   kernel_initializer=initializer, use_bias=False))
        result.add(InstanceNormalization())
        if apply_dropout:
            result.add(tf.keras.layers.Dropout(0.5))
        result.add(tf.keras.layers.ReLU())
        return result

    def unet_generator(self):
        down_stack = [
            self.downsample(64, 4, False),
            self.downsample(128, 4),
            self.downsample(128, 4),
            self.downsample(128, 4),
            self.downsample(128, 4)
        ]
        up_stack = [
            self.upsample(128, 4, True),
            self.upsample(128, 4, True),
            self.upsample(128, 4),
            self.upsample(64, 4)
        ]

        initializer = tf.random_normal_initializer(0., 0.02)
        last = tf.keras.layers.Conv2DTranspose(3, 4, strides=2, padding='same', kernel_initializer=initializer,
                                               activation='tanh')
        concat = tf.keras.layers.Concatenate()
        inputs = tf.keras.layers.Input(shape=[256, 256, 3])
        x = inputs
        skips = []
        for down in down_stack:
            x = down(x)
            skips.append(x)
        skips = reversed(skips[:-1])
        for up, skip in zip(up_stack, skips):
            x = up(x)
            x = concat([x, skip])
        x = last(x)
        return tf.keras.Model(inputs=inputs, outputs=x)


class Discriminator(unet_model):
    def __init__(self):
        self.model = self.discriminator()

    def discriminator(self):
        initializer = tf.random_normal_initializer(0., 0.02)
        inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
        x = inp
        down1 = self.downsample(64, 4, False)(x)  # (bs, 16, 16, 64)
        down2 = self.downsample(128, 4)(down1)
        down3 = self.downsample(256, 4)(down2)
        zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3)  # (bs, 34, 34, 256)
        conv = tf.keras.layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer,
                                      use_bias=False)(zero_pad1)  # (bs, 31, 31, 512)
        norm1 = InstanceNormalization()(conv)
        leaky_relu = tf.keras.layers.LeakyReLU()(norm1)
        zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 33, 512)
        last = tf.keras.layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 30, 1)
        return tf.keras.Model(inputs=inp, outputs=last)

class old_squeeze():
    def __init__(self):
        self.model = self.squeeze_attention_unet()

    def conv_block(self, x, num_filters):
        x = L.Conv2D(num_filters, 3, padding="same")(x)
        x = tfal.InstanceNormalization(axis=-1)(x)
        x = L.Activation("relu")(x)

        x = L.Conv2D(num_filters, 3, padding="same")(x)
        x = tfal.InstanceNormalization(axis=-1)(x)
        x = L.Activation("relu")(x)
        return x

    def se_block(self, x, num_filters, ratio=8):
        se_shape = (1, 1, num_filters)
        se = L.GlobalAveragePooling2D()(x)
        se = L.Reshape(se_shape)(se)
        se = L.Dense(num_filters // ratio, activation="relu", use_bias=False)(se)
        se = L.Dense(num_filters, activation="sigmoid", use_bias=False)(se)
        se = L.Reshape(se_shape)(se)
        x = L.Multiply()([x, se])
        return x

    def encoder_block(self, x, num_filters):
        x = self.conv_block(x, num_filters)
        x = self.se_block(x, num_filters)
        p = L.MaxPool2D((2, 2))(x)
        return x, p

    def decoder_block(self, x, s, num_filters):
        x = L.UpSampling2D(interpolation="bilinear")(x)
        x = L.Concatenate()([x, s])
        x = self.conv_block(x, num_filters)
        x = self.se_block(x, num_filters)
        return x

    def squeeze_attention_unet(self, input_shape=(256, 256, 3)):
        """ Inputs """
        inputs = L.Input(input_shape)

        """ Encoder """
        s1, p1 = self.encoder_block(inputs, 64)
        s2, p2 = self.encoder_block(p1, 128)
        s3, p3 = self.encoder_block(p2, 256)

        b1 = self.conv_block(p3, 512)
        b1 = self.se_block(b1, 512)

        """ Decoder """
        d1 = self.decoder_block(b1, s3, 256)
        d2 = self.decoder_block(d1, s2, 128)
        d3 = self.decoder_block(d2, s1, 64)

        """ Outputs """
        outputs = L.Conv2D(3, (1, 1), activation='tanh')(d3)

        """ Model """
        model = Model(inputs, outputs, name="Squeeze-Attention-UNET")
        return model

In [ ]:
def preprocess_image_train(image):
    image = (image/127.5)-1
    return image

# TO DO: Generalize Path, add it to arguments
def generate_images_GIF(img_input, model, img_true, mode, order):
    prediction = model(img_input)
    pred_vol = prediction[0, :, :, 0].numpy().copy()
    error = tf.image.ssim(img_true, prediction, max_val=2)
    img_input = np.rot90(img_input[0, :, :, 0], 3)
    img_true = np.rot90(img_true[0, :, :, 0], 3)
    prediction = np.rot90(prediction[0, :, :, 0], 3)

    plt.figure(figsize=(10, 6))
    if mode == 1:
        display_list = [img_input, prediction, img_true]
        title = [f'{seq_1} True', f'{seq_2} predicted', f'{seq_2} True']

    else:
        display_list = [img_input, prediction, img_true]
        title = [f'{seq_2} True', f'{seq_1} predicted', f'{seq_1} True']

    plots_path_T1_FLAIR = r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF'.format(brats_num, seq_1,
                                                                                                       seq_2)
    plots_path_FLAIR_T1 = r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF'.format(brats_num, seq_2,
                                                                                                       seq_1)
    if not os.path.exists(plots_path_T1_FLAIR):
        os.makedirs(plots_path_T1_FLAIR)
    if not os.path.exists(plots_path_FLAIR_T1):
        os.makedirs(plots_path_FLAIR_T1)

    for i in range(3):
        plt.subplot(1, 3, i + 1)
        plt.title(title[i])
        plt.imshow(display_list[i] * 0.5 + 0.5, cmap='gray')
        plt.axis('off')
        if mode == 1:
            plt.savefig(
                r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF\{}.png'.format(brats_num, seq_1,
                                                                                                    seq_2, order))
        if mode == 2:
            plt.savefig(
                r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF\{}.png'.format(brats_num, seq_2,
                                                                                                    seq_1, order))
    plt.show()
    return error, pred_vol


#TO DO: Generalize Path, add it to arguments
def predict_image(img_input, model):
    prediction = model(img_input)
    pred_vol = prediction[0, :, :, 0].numpy().copy()
    return pred_vol

#TO DO: Generalize Path, add it to arguments
def predict_image_and_calc_loss(img_input, model, img_true):
    prediction = model(img_input)
    pred_vol = prediction[0, :, :, 0].numpy().copy()
    # error = tf.image.ssim(img_true, prediction, max_val=2)
    return 0, pred_vol

def black_seq_generator(test_path, brats_num, T1_FLAG=True, T2_FLAG=True, FLAIR_FLAG=True):
    test_data_list = sorted(glob.glob(test_path + '/*'))
    original_vol_path = sorted(glob.glob(test_path + '/*'))[0]
    original_vol = nib.load(original_vol_path)
    original_shape = original_vol.shape

    v = np.zeros(original_shape)
    v = nib.Nifti1Image(v, original_vol.affine)  # to save this 3D (ndarry) numpy

    if FLAIR_FLAG:
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_flair.nii.gz')
    if T1_FLAG:
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_t1.nii.gz')
    if T2_FLAG:
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_t2.nii.gz')

    if T1_FLAG and T2_FLAG:
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_t1.nii.gz')
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_t2.nii.gz')
    if T1_FLAG and FLAIR_FLAG:
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_t1.nii.gz')
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_flair.nii.gz')
    if T2_FLAG and FLAIR_FLAG:
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_flair.nii.gz')
        nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_t2.nii.gz')
    print("Done")

def copy_subfolders_into_another_folder(paths_txt, source_folder, destination_folder):
    # Read the contents of the file
    with open(paths_txt, 'r') as file:
        contents = file.read()

    # Replace single quotes with double quotes
    contents = contents.replace("'", "\"")

    # Load the JSON array, the list of subfolder names to copy
    subfolder_names = json.loads(contents)

    # Loop through each item in the source folder
    for item in os.listdir(source_folder):
        # If the item is a subfolder and its name is in the list
        if os.path.isdir(os.path.join(source_folder, item)) and item in subfolder_names:
            # Copy the subfolder to the destination folder
            shutil.copytree(os.path.join(source_folder, item), os.path.join(destination_folder, item))

    copy_subfolders_into_another_folder

# The following function is responsible for returning the indices of the brain of the volume that contains foreground voxels.
def find_brain_width_wise(dep, hei, i, img):        #cropping width wise
    slice2D = img.get_fdata()[:, i, :]
    for j in range(hei):
        for k in range(dep):
            if slice2D[j, k] != 0:
                return i
    return 0

def find_brain_height_wise(dep, wid, i, img):      #cropping height wise
    slice2D = img.get_fdata()[i, :, :]
    for j in range(wid):
        for k in range(dep):
            if slice2D[j, k] != 0:
                return i
    return 0

def find_brain_depth_wise(wid, hei, i, img):        #cropping depth wise
    slice2D = img.get_fdata()[:, :, i]
    for j in range(wid):
        for k in range(hei):
            if slice2D[j, k] != 0:
                return i
    return 0


### Reading Cropped BraTS21 examples, change the root_path to you base address

In [ ]:
# !unzip '/content/drive/MyDrive/GP/T1c Validation.zip' -d '/content/Synthesized BraTS Validation Brain Cropped'
!unzip '/content/drive/MyDrive/GP/Synthesized BraTS Validation Brain Cropped.zip'

In [ ]:
'''
Just for Validation Set of BraTS
'''


root_path = '/content/drive/MyDrive/GP/BraTS Validation Brain Cropped'
data_list = sorted(glob.glob(root_path + '/*'))                            #list of paths of the inside files
# results_path = 'Synthesized BraTS Validation Brain Cropped/T1c Validation' #'Synthesized BraTS Validation Brain Cropped'
results_path = 'Synthesized BraTS Validation Brain Cropped'

In [ ]:
len(sorted(glob.glob(results_path + '/*')))

251

In [ ]:
# brats_num = input("please enter the brats number  ")
# brats_num = '028'
seq_1 = 'T2'
seq_2 = 'FLAIR'

In [ ]:
!unzip '/content/drive/MyDrive/GP/FINAL MODELS/T1---T1c/last.zip' -d "/content/T1T1c Model"

Archive:  /content/drive/MyDrive/GP/FINAL MODELS/T1---T1c/last.zip
  inflating: /content/T1T1c Model/last/checkpoint  
  inflating: /content/T1T1c Model/last/ckpt-54.data-00000-of-00001  
  inflating: /content/T1T1c Model/last/ckpt-54.index  


In [ ]:
'''
FOR GANs ONLY NOT CYCLE-GANS
'''

def downsample(filters, size, apply_norm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                                               kernel_initializer=initializer, use_bias=False))

    if apply_norm:
        result.add(tfal.InstanceNormalization(axis=-1))
    result.add(tf.keras.layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                               kernel_initializer=initializer, use_bias=False))
    result.add(tfal.InstanceNormalization(axis=-1))
    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))
    result.add(tf.keras.layers.ReLU())

    return result

def conv_block(x, num_filters):
    x = L.Conv2D(num_filters, 3, padding="same")(x)
    x = tfal.InstanceNormalization(axis=-1)(x)
    x = L.Activation("relu")(x)

    x = L.Conv2D(num_filters, 3, padding="same")(x)
    x = tfal.InstanceNormalization(axis=-1)(x)
    x = L.Activation("relu")(x)

    return x


def se_block(x, num_filters, ratio=8):
    se_shape = (1, 1, num_filters)
    se = L.GlobalAveragePooling2D()(x)
    se = L.Reshape(se_shape)(se)
    se = L.Dense(num_filters // ratio, activation="relu", use_bias=False)(se)
    se = L.Dense(num_filters, activation="sigmoid", use_bias=False)(se)
    se = L.Reshape(se_shape)(se)
    x = L.Multiply()([x, se])
    return x

def encoder_block(x, num_filters):
    x = conv_block(x, num_filters)
    x = se_block(x, num_filters)
    p = L.MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(x, s, num_filters):
    x = L.UpSampling2D(interpolation="bilinear")(x)
    x = L.Concatenate()([x, s])
    x = conv_block(x, num_filters)
    x = se_block(x, num_filters)
    return x

def squeeze_attention_unet(input_shape=(256, 256, 3)):
    """ Inputs """
    inputs = L.Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)


    b1 = conv_block(p4, 1024)
    b1 = se_block(b1, 1024)


    """ Decoder """
    d =  decoder_block(b1, s4, 512)
    d1 = decoder_block(d, s3, 256)
    d2 = decoder_block(d1, s2, 128)
    d3 = decoder_block(d2, s1, 64)

    """ Outputs """
    outputs = L.Conv2D(3, (1, 1), activation='tanh')(d3)

    """ Model """

    model = Model(inputs, outputs, name="Squeeze-Attention-UNET")
    return model

def discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    inp = tf.keras.layers.Input(shape=[256, 256, 3], name='input_image')
    x = inp
    down1 = downsample(64, 4, False)(x) # (bs, 16, 16, 64)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)


    zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
    conv = tf.keras.layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer,
                                  use_bias=False)(zero_pad1) # (bs, 31, 31, 512)
    norm1 = tfal.InstanceNormalization()(conv)
    leaky_relu = tf.keras.layers.LeakyReLU()(norm1)
    zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

    last = tf.keras.layers.Conv2D(3, 4, strides=1, kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)
    return tf.keras.Model(inputs=inp, outputs=last)

generator_g = squeeze_attention_unet() # Generates T2 images using T1 MRI Scans
discriminator_x = discriminator()
generator_g_optimizer = tf.keras.optimizers.legacy.Adam(2e-10, beta_1=0.5 )
discriminator_x_optimizer = tf.keras.optimizers.legacy.Adam(2e-10, beta_1=0.5 )

checkpoint_path = "/content/T1T1c Model/last"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           discriminator_x=discriminator_x,
                           generator_g_optimizer=generator_g_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=300)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print(f'Last Check Point: {ckpt_manager.latest_checkpoint}')
    print('Latest checkpoint restored!!')

In [ ]:
# !unzip '/content/drive/MyDrive/GP/Models/cp-25-Gans.zip'

# Predict

In [ ]:
# generator_g = ResNet().model
# generator_f = ResNet().model

# discriminator_x = Discriminator().model
# discriminator_y = Discriminator().model

# generator_g_optimizer = tf.keras.optimizers.legacy.Adam(8e-5)
# generator_f_optimizer = tf.keras.optimizers.legacy.Adam(8e-5)
# discriminator_x_optimizer = tf.keras.optimizers.legacy.Adam(8e-5)
# discriminator_y_optimizer = tf.keras.optimizers.legacy.Adam(8e-5)

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
print("Generator's parameters = {:,}".format(generator_g.count_params()))
print("Discriminator's parameters = {:,}".format(discriminator_x.count_params()))

Generator's parameters = 31,827,075
Discriminator's parameters = 2,781,955


In [ ]:
#TO DO: Generalize Path, add it to arguments
def predict_image(img_input, model, img_true):
    prediction = model(img_input)
    pred_vol = prediction[0, :, :, 0].numpy().copy()
    # error = tf.image.ssim(img_true, prediction, max_val=2)
    return 0, pred_vol

In [ ]:
# !unzip '/content/drive/MyDrive/GP/Models/T2-F 0-30.zip'
!unzip '/content/drive/MyDrive/GP/Models/FLAIR-T2-71.zip'
# !unzip '/content/drive/MyDrive/GP/Models/0-30-squeez.zip'

Archive:  /content/drive/MyDrive/GP/Models/FLAIR-T2-71.zip
  inflating: FLAIR-T2-71/checkpoint  
  inflating: FLAIR-T2-71/ckpt-71.data-00000-of-00001  
  inflating: FLAIR-T2-71/ckpt-71.index  


In [ ]:
# checkpoint_path = r"E:\Graduation Project\GIFs and Models\Attention UNET-1"
# checkpoint_path = r"E:\Graduation Project\GIFs and Models\259 CheckPoint 17-2-2023-T1-FLAIR"
# checkpoint_path = r"E:\Graduation Project\GIFs and Models\ResNet-7 7-3-2023 T2-Flair-71"
checkpoint_path = '/content/FLAIR-T2-71'
# checkpoint_path = '0-30-squeez'

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    # ckpt.restore(ckpt_manager.checkpoints[30-1])
#     ckpt.restore(ckpt_manager.latest_checkpoint)
    # print(f'Last Check Point: {ckpt_manager.checkpoints[30-1]} is restored')
    print(f'Last Check Point: {ckpt_manager.latest_checkpoint} is restored')

Last Check Point: /content/FLAIR-T2-71/ckpt-71 is restored


In [ ]:
with open('BraTS Validation on SqzAtt T1-T2 30 epochs.log', 'w') as file:
    file.write('')

In [ ]:
import logging

# Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create a file handler and set the formatter
file_handler = logging.FileHandler('/content/drive/MyDrive/GP/Models/BraTS Validation New on SqzAtt T1-T2 30 epochs.log')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('Brats %(iteration)s - T1-FLAIR ssim = %(ssim_score).4f')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Copy from root to result

In [ ]:
import glob

In [ ]:
root_path

'/content/drive/MyDrive/GP/BraTS Validation Brain Cropped'

In [ ]:
x = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-T1c From T1 - 29 July 2023'

# Get a list of all the subfolders in the source folder
subfolders = [f.path for f in os.scandir(root_path) if f.is_dir()]

# # Copy each subfolder to the two destination folders
for folder in subfolders:
    shutil.copytree(folder, os.path.join(x, os.path.basename(folder)))

print(len(glob.glob(x + '/*')))

total_files = 0
for root, dirs, files in os.walk(x):
    total_files += len(files)
print('Total number of files in folder and subfolders:', total_files)

In [ ]:
# root_path = '/content/drive/MyDrive/GP/BraTS Validation Brain Cropped'
# results_path = r'E:\Graduation Project\Datasets\Synthesized Brats 200-300 BC'
results_data_list = sorted(glob.glob(results_path + '/*'))

# T1_syn_result_vol_path = '/content/drive/MyDrive/GP/T2-FLAIR/ResNet-7(71 ep) - T1 Synth from FLAIR - BraTS Validation - Brain Cropped'
# T2_syn_result_vol_path = '/content/drive/MyDrive/GP/FLAIR From T1 Using GANS (SSIM) - 31 May 2023'
FLAIR_syn_result_vol_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-T1c From T1 - 29 July 2023'

# Get a list of all the subfolders in the source folder
subfolders = [f.path for f in os.scandir(root_path) if f.is_dir()]

# # Copy each subfolder to the two destination folders
# for folder in subfolders:
    # shutil.copytree(folder, os.path.join(T1_syn_result_vol_path, os.path.basename(folder)))
    # shutil.copytree(folder, os.path.join(T2_syn_result_vol_path, os.path.basename(folder)))
    # shutil.copytree(folder, os.path.join(FLAIR_syn_result_vol_path, os.path.basename(folder)))

# T1_syn_result_vols = sorted(glob.glob(T1_syn_result_vol_path + '/*'))
# T2_syn_result_vols = sorted(glob.glob(T2_syn_result_vol_path + '/*'))
FLAIR_syn_result_vols = sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))

In [ ]:
results_data_list

In [ ]:
T1_syn_result_vol_path = '/content/drive/MyDrive/GP/ResNet7(30 ep) - T1 Synth from FLAIR - BraTS Validation - Brain Cropped'
# T2_syn_result_vol_path = '/content/drive/MyDrive/GP/T2 Synthesized BraTS Validation Brain Cropped/T2 Synthesized BraTS Validation Brain Cropped'
FLAIR_syn_result_vol_path = '/content/drive/MyDrive/GP/ResNet7(30 ep) - FLAIR Synth from T1 - BraTS Validation - Brain Cropped'


T1_syn_result_vols = sorted(glob.glob(T1_syn_result_vol_path + '/*'))
FLAIR_syn_result_vols = sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))

print(len(T1_syn_result_vols), len(FLAIR_syn_result_vols))

251 251


In [ ]:
for i in (sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))):
  x = len( sorted(glob.glob(i + '/*')))
  if x != 5:
    print(i)

print(len(FLAIR_syn_result_vols))

251


In [ ]:
FLAIR_syn_result_vol_path

'/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1c - 29 July 2023'

# Delete T1

In [ ]:
for dirpath, dirnames, filenames in os.walk(T1_syn_result_vol_path):
    for filename in filenames:
        if filename.endswith('t1.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)
#             print(f"{file_path} has been deleted.")

# Delete T2

In [ ]:
for dirpath, dirnames, filenames in os.walk(T2_syn_result_vol_path):
    for filename in filenames:
        if filename.endswith('t2.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)
#             print(f"{file_path} has been deleted.")

# Delete FLAIR

In [ ]:
for dirpath, dirnames, filenames in os.walk(FLAIR_syn_result_vol_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)

#Delete T1c

In [ ]:
for dirpath, dirnames, filenames in os.walk(FLAIR_syn_result_vol_path):
    for filename in filenames:
        if filename.endswith('t1ce.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)

In [ ]:
for i in (sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))):
  x = len( sorted(glob.glob(i + '/*')))
  if x != 4:
    print(i)

In [ ]:
len(sorted(glob.glob(FLAIR_syn_result_vol_path + '/*')))

251

In [ ]:
total_files = 0
for root, dirs, files in os.walk(FLAIR_syn_result_vol_path):
    total_files += len(files)

print('Total number of files in folder and subfolders:', total_files)

Total number of files in folder and subfolders: 1004


In [ ]:
SSIM1 = []
SSIM2 = []

vol_1 = []
vol_2 = []

ssim_1_list = []
ssim_2_list = []

order_1 = 0
order_2 = 0

ssim_score_1 = 0
ssim_score_2 = 0

seq_1 = 'T2'
seq_2 = 'FLAIR'

In [ ]:
# results_data_list = results_data_list[246:]

In [ ]:
results_data_list

In [ ]:
os.path.basename(sorted(glob.glob(data_list[0] + '/*'))[3])

'BraTS2021_00000_t1ce.nii.gz'

In [ ]:
'''
for FLAIR GANS ONLYYY
'''
def predict_image_and_NO_loss(img_input, model):
    prediction = model(img_input)
    pred_vol = prediction[0, :, :, 0].numpy().copy()
    return pred_vol

AUTOTUNE = tf.data.experimental.AUTOTUNE

for i, path in enumerate(results_data_list):
    T1_path = os.path.join(path, "images", "GIF T1")
    dep = len(glob.glob(T1_path + '/**/*'))

    GIF_T1 = tf.keras.preprocessing.image_dataset_from_directory(
                                  T1_path,
                                  seed=123,
                                  image_size=(256, 256),
                                  batch_size=1,
                                  shuffle = False)

    GIF_T1 = GIF_T1.cache().prefetch(buffer_size=AUTOTUNE)
    GIF_T1 = GIF_T1.map(lambda x, _: (preprocess_image_train(x)))

    for image_x in GIF_T1:
        img_1 = predict_image_and_NO_loss(image_x, generator_g)
        vol_2.append(img_1)

    original_vol_path = sorted(glob.glob(data_list[i] + '/*'))[0]
    original_vol = nib.load(original_vol_path)
    original_shape = original_vol.shape

    vol_2 = np.array(vol_2).transpose(1, 2, 0)

    vol_2 = ndimage.zoom(vol_2, (original_shape[0]/vol_2.shape[0],
                                 original_shape[1]/vol_2.shape[1],
                                 original_shape[2]/vol_2.shape[2]), order=0)

    v2 = nib.Nifti1Image(np.array(vol_2), original_vol.affine)
    # FLAIR_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[0])  #for flair
    FLAIR_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[3])   #for T1ce

    FLAIR_res_path = os.path.join(FLAIR_syn_result_vols[i], FLAIR_name)
    nib.save(v2, FLAIR_res_path)

    vol_1 = []
    vol_2 = []
    print(f"Volume #{i} is done")

In [ ]:
import shutil
import os

source_folder_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-T1c From T1 - 29 July 2023'
destination_folder_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/Zipped'

# Specify the name for the zip file
zip_filename = os.path.basename(source_folder_path)

# Create the full path for the zip file
zip_filepath = os.path.join(destination_folder_path, zip_filename)

# Create the zip file
shutil.make_archive(zip_filepath, 'zip', source_folder_path)

print("Zipping complete.")

In [ ]:
#DELETE THIS CELL THIS WAS BECAUSE YOU WERE TOO SLEEPY AND JUST WANTED TO MAKE MORE PREDICTIONS WHILE SLEEPING

!unzip '/content/drive/MyDrive/GP/FINAL MODELS/T1c--FLAIR/LAST.zip' -d "/content/T1c Model"
del generator_g
del discriminator_x
del generator_g_optimizer
del discriminator_x_optimizer

generator_g = squeeze_attention_unet()
discriminator_x = discriminator()
generator_g_optimizer = tf.keras.optimizers.legacy.Adam(2e-10, beta_1=0.5 )
discriminator_x_optimizer = tf.keras.optimizers.legacy.Adam(2e-10, beta_1=0.5 )

checkpoint_path = "/content/T1c Model/LAST"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           discriminator_x=discriminator_x,
                           generator_g_optimizer=generator_g_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=300)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print(f'Last Check Point: {ckpt_manager.latest_checkpoint}')
    print('Latest checkpoint restored!!')

print("Generator's parameters = {:,}".format(generator_g.count_params()))
print("Discriminator's parameters = {:,}".format(discriminator_x.count_params()))

xx = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1c - 29 July 2023'
subfolders = [f.path for f in os.scandir(root_path) if f.is_dir()]
for folder in subfolders:
    shutil.copytree(folder, os.path.join(xx, os.path.basename(folder)))
print(len(glob.glob(xx + '/*')))

total_files = 0
for root, dirs, files in os.walk(xx):
    total_files += len(files)
print('Total number of files in folder and subfolders:', total_files)

results_data_list = sorted(glob.glob(results_path + '/*'))
FLAIR_syn_result_vol_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1c - 29 July 2023'
subfolders = [f.path for f in os.scandir(root_path) if f.is_dir()]
FLAIR_syn_result_vols = sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))

for i in (sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))):
  x = len( sorted(glob.glob(i + '/*')))
  if x != 5:
    print(i)

print("before  cropping: ")
print(len(FLAIR_syn_result_vols))

for dirpath, dirnames, filenames in os.walk(FLAIR_syn_result_vol_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            os.remove(file_path)

for i in (sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))):
  x = len( sorted(glob.glob(i + '/*')))
  if x != 4:
    print(i)

print("after  cropping: ")
print(len(FLAIR_syn_result_vols))

total_files = 0
for root, dirs, files in os.walk(FLAIR_syn_result_vol_path):
    total_files += len(files)

print('Total number of files in folder and subfolders:', total_files)

SSIM1 = []
SSIM2 = []

vol_1 = []
vol_2 = []

ssim_1_list = []
ssim_2_list = []

order_1 = 0
order_2 = 0

ssim_score_1 = 0
ssim_score_2 = 0

seq_1 = 'T1c'
seq_2 = 'FLAIR'

AUTOTUNE = tf.data.experimental.AUTOTUNE

for i, path in enumerate(results_data_list):
    T1_path = os.path.join(path, "images", "GIF T1c")
    dep = len(glob.glob(T1_path + '/**/*'))

    GIF_T1 = tf.keras.preprocessing.image_dataset_from_directory(
                                  T1_path,
                                  seed=123,
                                  image_size=(256, 256),
                                  batch_size=1,
                                  shuffle = False)

    GIF_T1 = GIF_T1.cache().prefetch(buffer_size=AUTOTUNE)
    GIF_T1 = GIF_T1.map(lambda x, _: (preprocess_image_train(x)))

    for image_x in GIF_T1:
        img_1 = predict_image_and_NO_loss(image_x, generator_g)
        vol_2.append(img_1)

    original_vol_path = sorted(glob.glob(data_list[i] + '/*'))[0]
    original_vol = nib.load(original_vol_path)
    original_shape = original_vol.shape

    vol_2 = np.array(vol_2).transpose(1, 2, 0)

    vol_2 = ndimage.zoom(vol_2, (original_shape[0]/vol_2.shape[0],
                                 original_shape[1]/vol_2.shape[1],
                                 original_shape[2]/vol_2.shape[2]), order=0)

    v2 = nib.Nifti1Image(np.array(vol_2), original_vol.affine)
    FLAIR_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[0])
    FLAIR_res_path = os.path.join(FLAIR_syn_result_vols[i], FLAIR_name)
    nib.save(v2, FLAIR_res_path)

    vol_1 = []
    vol_2 = []
    print(f"Volume #{i} is done")

In [ ]:
import shutil
import os

source_folder_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T2 - 29 July 2023'
destination_folder_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/Zipped'
zip_filename = 'FINAL-GANS-FLAIR From T2 - 29 July 2023'
zip_filepath = os.path.join(destination_folder_path, zip_filename)
shutil.make_archive(zip_filepath, 'zip', source_folder_path)
print("Zipping complete.")

source_folder_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1 - 29 July 2023'
# destination_folder_path = '/content/drive/MyDrive/GP/Zip Synth Datasets/GANs-55-FLAIR from (T1, T2, T1c)'
zip_filename = 'FINAL-GANS-FLAIR From T1 - 29 July 2023'
zip_filepath = os.path.join(destination_folder_path, zip_filename)
shutil.make_archive(zip_filepath, 'zip', source_folder_path)
print("Zipping complete.")

source_folder_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1c - 29 July 2023'
# destination_folder_path = '/content/drive/MyDrive/GP/Zip Synth Datasets/GANs-55-FLAIR from (T1, T2, T1c)'
zip_filename = 'FINAL-GANS-FLAIR From T1c - 29 July 2023'
zip_filepath = os.path.join(destination_folder_path, zip_filename)
shutil.make_archive(zip_filepath, 'zip', source_folder_path)
print("Zipping complete.")

Zipping complete.
Zipping complete.
Zipping complete.


In [ ]:
results_data_list[0]

'Synthesized BraTS Validation Brain Cropped/BraTS2021_00000'

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

for i, path in enumerate(results_data_list):
    T1_path = os.path.join(path, "images", "GIF T1")
    # T2_path = os.path.join(path, "images", "GIF T2")
    FLAIR_path = os.path.join(path, "images", "GIF Flair") ###### check this T1c instead of FLAIR
    # T1c_path = os.path.join(path, "images", "GIF T1c")

    dep = len(glob.glob(FLAIR_path + '/**/*'))

    GIF_T1 = tf.keras.preprocessing.image_dataset_from_directory(
                                  T1_path,
                                  seed=123,
                                  image_size=(256, 256),
                                  batch_size=1,
                                  shuffle = False)

    # GIF_T2 = tf.keras.preprocessing.image_dataset_from_directory(
    #                               T2_path,
    #                               seed=123,
    #                               image_size=(256, 256),
    #                               batch_size=1,
    #                               shuffle = False)

    GIF_FLAIR = tf.keras.preprocessing.image_dataset_from_directory(
                                  FLAIR_path,
                                  seed=123,
                                  image_size=(256, 256),
                                  batch_size=1,
                                  shuffle = False)

    # GIF_T1c = tf.keras.preprocessing.image_dataset_from_directory(
    #                               T1c_path,
    #                               seed=123,
    #                               image_size=(256, 256),
    #                               batch_size=1,
    #                               shuffle = False)

    GIF_T1 = GIF_T1.cache().prefetch(buffer_size=AUTOTUNE)
    # GIF_T2 = GIF_T2.cache().prefetch(buffer_size=AUTOTUNE)
    GIF_FLAIR = GIF_FLAIR.cache().prefetch(buffer_size=AUTOTUNE)
    GIF_T1c = GIF_T1c.cache().prefetch(buffer_size=AUTOTUNE)

    # GIF_T1 = GIF_T1.map(lambda x, _: (preprocess_image_train(x)))
    # GIF_T2 = GIF_T2.map(lambda x, _: (preprocess_image_train(x)))
    GIF_FLAIR = GIF_FLAIR.map(lambda x, _: (preprocess_image_train(x)))
    GIF_T1c = GIF_T1c.map(lambda x, _: (preprocess_image_train(x)))


    for image_x, image_y in tf.data.Dataset.zip((GIF_T1c, GIF_FLAIR)):
        ssim_1, img_1 = predict_image_and_calc_loss(image_x, generator_g, image_y) #predict_image_and_calc_loss
        ssim_score_1 = ssim_score_1 + ssim_1
        ssim_2_list.append(ssim_1)
        vol_2.append(img_1)

    # for image_x, image_y in tf.data.Dataset.zip((GIF_T1, GIF_FLAIR)):
    #     ssim_2, img_2 = predict_image_and_calc_loss(image_y, generator_f, image_x)
    #     ssim_score_2 = ssim_score_2 + ssim_2
    #     ssim_1_list.append(ssim_2)
    #     vol_1.append(img_2)

    ssim_score_1, ssim_score_2 = ssim_score_1/dep , ssim_score_2/dep
    # ssim_score_1 = ssim_score_1/dep
    # logger.info('', extra={'iteration': i+1, 'ssim_score': ssim_score_1.numpy()[0]})
    # logger.info('', extra={'iteration': i+1, 'ssim_score': ssim_score_2.numpy()[0]})


    original_vol_path = sorted(glob.glob(data_list[i] + '/*'))[0]
    original_vol = nib.load(original_vol_path)
    original_shape = original_vol.shape


    # vol_1 = np.array(vol_1).transpose(1, 2, 0)
    vol_2 = np.array(vol_2).transpose(1, 2, 0)

    # vol_1 = ndimage.zoom(vol_1, (original_shape[0]/vol_1.shape[0],
    #                              original_shape[1]/vol_1.shape[1],
    #                              original_shape[2]/vol_1.shape[2]), order=0)

    vol_2 = ndimage.zoom(vol_2, (original_shape[0]/vol_2.shape[0],
                                 original_shape[1]/vol_2.shape[1],
                                 original_shape[2]/vol_2.shape[2]), order=0)


    # v1 = nib.Nifti1Image(np.array(vol_1), original_vol.affine)            # to save this 3D (ndarry) numpy
    v2 = nib.Nifti1Image(np.array(vol_2), original_vol.affine)            # to save this 3D (ndarry) numpy

    # T1_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[2])
    # T2_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[4])
    FLAIR_name = os.path.basename(sorted(glob.glob(data_list[i] + '/*'))[0])

    # T1_res_path = os.path.join(T1_syn_result_vols[i], T1_name)
    # T2_res_path = os.path.join(T2_syn_result_vols[i], T2_name)
    FLAIR_res_path = os.path.join(FLAIR_syn_result_vols[i], FLAIR_name)


    # Save the Nifti image to a file
    # nib.save(v1, T1_res_path)
    # nib.save(v2, T2_res_path)
    nib.save(v2, FLAIR_res_path)


    vol_1 = []
    vol_2 = []

    SSIM1.append(ssim_score_1)
    SSIM2.append(ssim_score_2)

    ssim_score_1 = 0
    ssim_score_2 = 0
    print(f"Volume #{i} is done")

In [ ]:
SSIM1

[0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
for i in (sorted(glob.glob(FLAIR_syn_result_vol_path + '/*'))):
  x = len( sorted(glob.glob(i + '/*')))
  if x != 5:
    print(i)

len(sorted(glob.glob(FLAIR_syn_result_vol_path + '/*')))

251

In [ ]:
T2_res_path = os.path.join(T2_syn_result_vols[i], T2_name)
FLAIR_res_path = os.path.join(FLAIR_syn_result_vols[i], FLAIR_name)


# Save the Nifti image to a file
nib.save(v1, FLAIR_res_path)
#     nib.save(v2, T2_res_path)
nib.save(v2, T2_res_path)


vol_1 = []
vol_2 = []

ssim_score_1 = 0
ssim_score_2 = 0
print(f"Volume #{i} is done")

Volume #0 is done


In [ ]:
np.shape(vol_1)

(135, 256, 256)

In [ ]:
ssim_score_1, ssim_score_2 = ssim_score_1/dep , ssim_score_2/dep


print("ssim_score for first Conversion = {:.4f} ".format(ssim_score_1.numpy()[0]))
print("ssim_score for second Conversion = {:.4f} ".format(ssim_score_2.numpy()[0]))

ssim_score for first Conversion = 0.6095 
ssim_score for second Conversion = 0.6937 


In [ ]:
original_vol_path = sorted(glob.glob(root_path + '/*'))[0]
original_vol = nib.load(original_vol_path)
x = original_vol.get_fdata()
print(x.min(), x.max())



original_vol = nib.load(r'E:\Graduation Project\GIFs and Models\Brats 006\Testing Synthesis With Segmentation on BraTS #006\FLAIR.nii.gz')
x = original_vol.get_fdata()
print(x.min(), x.max())

0.0 3724.0
-1.0 0.9064252972602844


In [ ]:
original_vol_path = sorted(glob.glob(root_path + '/*'))[0]
original_vol = nib.load(original_vol_path)
original_shape = original_vol.shape

result_vol_path = r'E:\Graduation Project\GIFs and Models\Brats {}\Synthesized Volumes'.format(brats_num)
if not os.path.exists(result_vol_path):
    os.makedirs(result_vol_path)

vol_1 = np.array(vol_1).transpose(1, 2, 0)
vol_2 = np.array(vol_2).transpose(1, 2, 0)


vol_1 = ndimage.zoom(vol_1, (original_shape[0]/vol_1.shape[0],
                                             original_shape[1]/vol_1.shape[1],
                                             original_shape[2]/vol_1.shape[2]), order=0)

vol_2 = ndimage.zoom(vol_2, (original_shape[0]/vol_2.shape[0],
                                             original_shape[1]/vol_2.shape[1],
                                             original_shape[2]/vol_2.shape[2]), order=0)


v1 = nib.Nifti1Image(np.array(vol_1), original_vol.affine)            # to save this 3D (ndarry) numpy
v2 = nib.Nifti1Image(np.array(vol_2), original_vol.affine)            # to save this 3D (ndarry) numpy


# Save the Nifti image to a file
nib.save(v1, result_vol_path + '/' + 'T1.nii.gz')
nib.save(v2, result_vol_path + '/' + 'T2.nii.gz')

In [ ]:
anim_file = r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}.gif'.format(brats_num, seq_1, seq_2)

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob(r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF\*.png'.format(brats_num, seq_1, seq_2))
  filenames = sorted(filenames)
  filenames = list(Tcl().call('lsort', '-dict', filenames))
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

embed.embed_file(anim_file)

In [ ]:
anim_file = r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}.gif'.format(brats_num, seq_2, seq_1)

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob(r'E:\Graduation Project\GIFs and Models\Brats {}\Predicted\{}-{}-GIF\*.png'.format(brats_num, seq_2, seq_1))
  filenames = sorted(filenames)
  filenames = list(Tcl().call('lsort', '-dict', filenames))
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

embed.embed_file(anim_file)